In [1]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
import json

In [2]:
if __name__ == '__main__':
    
    spark_context = SparkSession.builder.appName("readApp").getOrCreate()
    
    file_path = "\\path_to_csv\\USCensus.csv"
    spark_df = spark_context.read.csv(file_path, header = True).cache()
    print("======== File Loaded Successfully =========")
    spark_df.show(5)
    print(" ======== Basic Analysis ==========" + "\n")
    print("There are " + str(spark_df.count()) + " records" + "\n")
    education = spark_df.groupby('education').count()
    print(education.show())
    print(" ======= Using SQL for EDA ========" + "\n")
    spark_df.registerTempTable("censusData")
    output = spark_context.sql("SELECT * from censusData where education = 'Masters'")
    output.show(5)
    print("\n" + "Got " + str(output.count()) + " records as output" + "\n")
    print(" ========== Saving Output in JSON format =============" + "\n")
    output.coalesce(1).write.format('json').save("output.json")
    
    

======== File Loaded Successfully =========
+---+----------------+-----------+---------+-------------+------------------+-----------------+-------------+-----+------+------------+------------+--------------+--------------+------+
|age|       workclass|demogweight|education|education-num|    marital-status|       occupation| relationship| race|   sex|capital-gain|capital-loss|hours-per-week|native-country|income|
+---+----------------+-----------+---------+-------------+------------------+-----------------+-------------+-----+------+------------+------------+--------------+--------------+------+
| 39|       State-gov|      77516|Bachelors|           13|     Never-married|     Adm-clerical|Not-in-family|White|  Male|        2174|           0|            40| United-States|<=50K.|
| 50|Self-emp-not-inc|      83311|Bachelors|           13|Married-civ-spouse|  Exec-managerial|      Husband|White|  Male|           0|           0|            13| United-States|<=50K.|
| 38|         Private|    

In [3]:
df = []

In [5]:
with open('\\path_to_output_folder\\file_name.json') as f:
    for obj in f:
        rec = json.loads(obj)
        df.append(rec)

In [6]:
df = pd.DataFrame(df)

In [7]:
df.describe()

,age,workclass,demogweight,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
count,1300,1300,1300,1300,1300,1300,1300,1300,1300,1300,1300,1300,1300,1300,1300
unique,61,7,1233,1,1,6,14,6,5,2,48,32,55,27,2
top,42,Private,280167,Masters,14,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,40,United-States,>50K.
freq,58,659,3,1300,1300,759,634,666,1167,892,1100,1183,503,1160,731
